In [ ]:
!pip install pandas numpy geopy scikit-learn

In [130]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler

## Clean and Convert Data

In [131]:
def count_items(x):
            if pd.isna(x):
                return 0
            return len(str(x).split(';'))

In [142]:
for district_num in range(1, 29):  
    try:  
        df = pd.read_csv(f"../datasets/updated_coordinates/district{district_num}.csv")

        # Remove commas, dollar signs, etc., and convert to numeric
        for col in ["Transacted Price ($)", "Area (SQFT)", "Unit Price ($ PSF)", "Area (SQM)", "Unit Price ($ PSM)"]:
            df[col] = df[col].replace(r'[\$,]', '', regex=True).astype(float)
            df = df.dropna(subset=["Unit Price ($ PSF)", "Area (SQFT)"])

        df = df[~df['Floor Level'].isin(['B1', 'B2', 'G', 'PH'])]
        df['Sale Date'] = pd.to_datetime(df['Sale Date'], format='%b-%y', errors='coerce')
        df['Sale Months Since Sep 2020'] = round((df['Sale Date'] - pd.Timestamp("2020-09-01")) / pd.Timedelta(days=30))

        df['Num MRT Within 1km'] = df['Nearest MRT Stations'].apply(count_items)
        df['Num Hawker Within 1km'] = df['Nearby Hawker Centers'].apply(count_items)
        df['Num Malls Within 1km'] = df['Shopping Malls Within Radius of 1km'].apply(count_items)
        df['Num Hospitals Within 5km'] = df['Hospitals Within Radius of 5km'].apply(count_items)
        df['Num Schools Within 2km'] = df['Schools Within Radius of 2km'].apply(count_items)
        df['Num Parks Within 1km'] = df['Parks Within Radius of 1km'].apply(count_items)

        df.drop(columns=['Nett Price($)', 'Street Name', 'Nearest MRT Stations', 'Nearby Hawker Centers', 'Shopping Malls Within Radius of 1km', 'Hospitals Within Radius of 5km', 'Schools Within Radius of 2km', 'Parks Within Radius of 1km', 'Number of Units'], inplace=True)

        df.to_csv(f"../datasets/cleaned_data/district{district_num}_cleaned.csv", index=False)

    except Exception as e:
        print(f"Error processing district {district_num}: {str(e)}")
        continue

Error processing district 24: [Errno 2] No such file or directory: '../datasets/updated_coordinates/district24.csv'


## Encode

In [140]:
# Convert 'Floor Level' to numerical average
def level_to_num(x):
    if pd.isna(x):
        return None
    parts = x.split(' to ')
    if len(parts) == 2:
        return (int(parts[0]) + int(parts[1])) / 2
    try:
        return int(x)
    except:
        return None

In [145]:
for district_num in range(1, 29):
    try:
        cleaned_df = pd.read_csv(f"../datasets/cleaned_data/district{district_num}_cleaned.csv")

        # Encode categorical data into numerical data using one-hot encoding
        categorical_cols = ["Type of Sale", "Property Type", "Tenure", "Market Segment", "Postal District"]
        encoded_cleaned_df = pd.get_dummies(cleaned_df, columns=categorical_cols, drop_first=True)

        encoded_cleaned_df['Avg_Floor_Level'] = encoded_cleaned_df['Floor Level'].apply(level_to_num)

        encoded_cleaned_df.to_csv(f"../datasets/clustering_data/district{district_num}_clustering.csv", index=False)
    
    except Exception as e:
        print(f"Error processing district {district_num}: {str(e)}")
        continue

Error processing district 24: [Errno 2] No such file or directory: '../datasets/cleaned_data/district24_cleaned.csv'


In [112]:
encoded_cleaned_df.head()

,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Type of Area,Area (SQM),Unit Price ($ PSM),Floor Level,Latitude,...,Tenure_99 yrs lease commencing from 2005,Tenure_99 yrs lease commencing from 2007,Tenure_99 yrs lease commencing from 2011,Tenure_99 yrs lease commencing from 2023,Tenure_99 yrs lease commencing from 2024,Tenure_999 yrs lease commencing from 1826,Tenure_999 yrs lease commencing from 1827,Tenure_Freehold,Market Segment_Rest of Central Region,Avg_Floor_Level
0,ONE MARINA GARDENS,1883403.0,656.60,2868.0,2025-09-01,Strata,61.0,30875.0,01 to 05,1.275801,...,False,False,False,True,False,False,False,False,True,3.0
1,ONE MARINA GARDENS,2055683.0,656.60,3131.0,2025-09-01,Strata,61.0,33700.0,36 to 40,1.275801,...,False,False,False,True,False,False,False,False,True,38.0
2,MARINA BAY RESIDENCES,1755000.0,731.95,2398.0,2025-09-01,Strata,68.0,25809.0,21 to 25,1.279626,...,True,False,False,False,False,False,False,False,False,23.0
3,ONE MARINA GARDENS,2038262.0,667.37,3054.0,2025-09-01,Strata,62.0,32875.0,26 to 30,1.275801,...,False,False,False,True,False,False,False,False,True,28.0
4,MARINA ONE RESIDENCES,4260000.0,2249.68,1894.0,2025-09-01,Strata,209.0,20383.0,01 to 05,1.277224,...,False,False,True,False,False,False,False,False,False,3.0


In [113]:
encoded_cleaned_df.columns.tolist()

['Project Name',
 'Transacted Price ($)',
 'Area (SQFT)',
 'Unit Price ($ PSF)',
 'Sale Date',
 'Type of Area',
 'Area (SQM)',
 'Unit Price ($ PSM)',
 'Floor Level',
 'Latitude',
 'Longitude',
 'Full Address',
 'Dist to CBD in Km',
 'Sale Months Since Sep 2020',
 'Num MRT Within 1km',
 'Num Hawker Within 1km',
 'Num Malls Within 1km',
 'Num Hospitals Within 5km',
 'Num Schools Within 2km',
 'Num Parks Within 1km',
 'Type of Sale_Resale',
 'Property Type_Condominium',
 'Tenure_99 yrs lease commencing from 1968',
 'Tenure_99 yrs lease commencing from 1970',
 'Tenure_99 yrs lease commencing from 1992',
 'Tenure_99 yrs lease commencing from 2002',
 'Tenure_99 yrs lease commencing from 2004',
 'Tenure_99 yrs lease commencing from 2005',
 'Tenure_99 yrs lease commencing from 2007',
 'Tenure_99 yrs lease commencing from 2011',
 'Tenure_99 yrs lease commencing from 2023',
 'Tenure_99 yrs lease commencing from 2024',
 'Tenure_999 yrs lease commencing from 1826',
 'Tenure_999 yrs lease commencin

## Z-Score Normalization

In [164]:
for district_num in range(1, 29):
    try:
        normalized_df = pd.read_csv(f"../datasets/clustering_data/district{district_num}_clustering.csv")

        num_cols = [
            "Transacted Price ($)", "Area (SQFT)", "Unit Price ($ PSF)",
            'Num MRT Within 1km',
            'Num Hawker Within 1km',
            'Num Malls Within 1km',
            'Num Hospitals Within 5km',
            'Num Schools Within 2km',
            'Num Parks Within 1km',
            "Dist to CBD in Km",
        ]

        scaler = StandardScaler()
        normalized_df[num_cols] = scaler.fit_transform(normalized_df[num_cols])

        normalized_df.to_csv(f"../datasets/clustering_data/district{district_num}_clustering.csv")
    
    except Exception as e:
        print(f"Error processing district {district_num}: {str(e)}")
        continue

Error processing district 24: [Errno 2] No such file or directory: '../datasets/clustering_data/district24_clustering.csv'


In [158]:
normalized_df.head()

,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Num MRT Within 1km,Num Hawker Within 1km,Num Malls Within 1km,Num Hospitals Within 5km,Num Schools Within 2km,Num Parks Within 1km,Avg_Floor_Level,Dist to CBD in Km,Latitude,Longitude
0,-0.211237,-0.460443,0.774350,-1.177227,-1.175060,-1.296450,-1.143547,-1.118512,-1.374335,-1.619747,1.347678,1.275801,103.862936
1,-0.087630,-0.460443,1.240198,-1.177227,-1.175060,-1.296450,-1.143547,-1.118512,-1.374335,0.947931,1.347678,1.275801,103.862936
2,-0.303362,-0.331436,-0.058154,-0.723616,-0.407922,0.640686,0.276567,-0.286674,0.161507,-0.152502,-0.618881,1.279626,103.854987
3,-0.100129,-0.442004,1.103809,-1.177227,-1.175060,-1.296450,-1.143547,-1.118512,-1.374335,0.214309,1.347678,1.275801,103.862936
4,1.493910,2.267081,-0.950882,0.183604,-0.407922,0.156402,-0.670176,-0.286674,-0.030473,-1.619747,-0.363513,1.277224,103.853695


In [159]:
normalized_df.columns.tolist()

['Transacted Price ($)',
 'Area (SQFT)',
 'Unit Price ($ PSF)',
 'Num MRT Within 1km',
 'Num Hawker Within 1km',
 'Num Malls Within 1km',
 'Num Hospitals Within 5km',
 'Num Schools Within 2km',
 'Num Parks Within 1km',
 'Avg_Floor_Level',
 'Dist to CBD in Km',
 'Latitude',
 'Longitude']

## Clustering Data

In [167]:
for district_num in range(1, 29):
    try:
        encoded_cleaned_df = pd.read_csv(f"../datasets/clustering_data/district{district_num}_clustering.csv")

        clustering_features = [
                'Transacted Price ($)',
                'Area (SQFT)',       # or 'Area (SQM)', not both
                'Unit Price ($ PSF)',# or 'Unit Price ($ PSM)'
                'Num MRT Within 1km',
                'Num Hawker Within 1km',
                'Num Malls Within 1km',
                'Num Hospitals Within 5km',
                'Num Schools Within 2km',
                'Num Parks Within 1km',
                'Dist to CBD in Km',
            ]

        cluster_df = encoded_cleaned_df[clustering_features]
        # Count truly empty or placeholder values
        print((cluster_df == '').sum())
        print((cluster_df == ' ').sum())
        print((cluster_df == 'NaN').sum())
        print((cluster_df == '-').sum())
    
        cluster_df.to_csv(f"../datasets/clustering_data/district{district_num}_clustering.csv", index=False)
        
    except Exception as e:
        print(f"Error processing district {district_num}: {str(e)}")
        continue

Transacted Price ($)        0
Area (SQFT)                 0
Unit Price ($ PSF)          0
Num MRT Within 1km          0
Num Hawker Within 1km       0
Num Malls Within 1km        0
Num Hospitals Within 5km    0
Num Schools Within 2km      0
Num Parks Within 1km        0
Dist to CBD in Km           0
dtype: int64
Transacted Price ($)        0
Area (SQFT)                 0
Unit Price ($ PSF)          0
Num MRT Within 1km          0
Num Hawker Within 1km       0
Num Malls Within 1km        0
Num Hospitals Within 5km    0
Num Schools Within 2km      0
Num Parks Within 1km        0
Dist to CBD in Km           0
dtype: int64
Transacted Price ($)        0
Area (SQFT)                 0
Unit Price ($ PSF)          0
Num MRT Within 1km          0
Num Hawker Within 1km       0
Num Malls Within 1km        0
Num Hospitals Within 5km    0
Num Schools Within 2km      0
Num Parks Within 1km        0
Dist to CBD in Km           0
dtype: int64
Transacted Price ($)        0
Area (SQFT)                 0
U

In [168]:
cluster_df.head()

,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Num MRT Within 1km,Num Hawker Within 1km,Num Malls Within 1km,Num Hospitals Within 5km,Num Schools Within 2km,Num Parks Within 1km,Dist to CBD in Km
0,-0.346776,0.175782,-0.765668,0.187207,-0.374606,1.438607,-1.442506,1.160223,-2.002701,2.382284
1,1.892528,1.835504,-0.098898,-1.238095,-0.374606,0.110836,0.451368,-0.126721,0.444833,-0.634930
2,0.558391,0.085257,0.435984,-0.287893,-0.374606,0.110836,0.451368,-1.413665,1.144128,-0.607620
3,0.391132,-0.850240,2.126113,1.612510,-0.374606,0.774722,0.451368,1.160223,0.444833,0.306465
4,-1.124038,-1.031289,-0.333366,2.087610,2.093339,2.102493,-1.442506,1.675000,-1.653053,0.927459


In [155]:
# Cleanliness check
print(df.isna().sum())  
print(df.dtypes)       

Project Name                  0
Transacted Price ($)          0
Area (SQFT)                   0
Unit Price ($ PSF)            0
Sale Date                     0
Type of Sale                  0
Type of Area                  0
Area (SQM)                    0
Unit Price ($ PSM)            0
Property Type                 0
Tenure                        0
Postal District               0
Market Segment                0
Floor Level                   0
Latitude                      0
Longitude                     0
Full Address                  0
Dist to CBD in Km             0
Sale Months Since Sep 2020    0
Num MRT Within 1km            0
Num Hawker Within 1km         0
Num Malls Within 1km          0
Num Hospitals Within 5km      0
Num Schools Within 2km        0
Num Parks Within 1km          0
dtype: int64
Project Name                          object
Transacted Price ($)                 float64
Area (SQFT)                          float64
Unit Price ($ PSF)                   float64
Sale Da

## Regression Data

In [79]:
X = df.drop(columns=["Unit Price ($ PSF)", "Project Name", "Full Address"])
y = df["Unit Price ($ PSF)"]